In [ ]:
# | default_exp nets/fpn_3d

# Imports

In [ ]:
# | export

from typing import Literal

import torch
from huggingface_hub import PyTorchModelHubMixin
from torch import nn
from torch.nn import functional as F

from vision_architectures.blocks.cnn import CNNBlock3D, CNNBlockConfig
from vision_architectures.utils.activation_checkpointing import ActivationCheckpointing
from vision_architectures.utils.custom_base_model import CustomBaseModel, Field, model_validator

# Config

In [ ]:
# | export


class FPN3DBlockConfig(CNNBlockConfig):
    dim: int
    skip_conn_dim: int
    is_deepest: bool = Field(False, description="True if this is the deepest block in the FPN, else False")
    interpolation_mode: str = "trilinear"
    merge_method: Literal["add", "concat"] = "add"

    in_channels: None = Field(None, description="Calculated based on other parameters")
    out_channels: None = Field(None, description="Calculated based on other parameters")


class FPN3DConfig(CustomBaseModel):
    blocks: list[FPN3DBlockConfig]

    @property
    def dim(self):
        return self.blocks[0].dim

    @classmethod
    def build(cls, skip_conn_dims: list[int], kernel_size: int = 3, **kwargs):
        blocks = []
        for i, skip_conn_dim in enumerate(skip_conn_dims):
            blocks.append(
                FPN3DBlockConfig(
                    skip_conn_dim=skip_conn_dim,
                    is_deepest=(i == len(skip_conn_dims) - 1),
                    kernel_size=kernel_size,
                    **kwargs
                )
            )
        return cls(blocks=blocks)

    @model_validator(mode="after")
    def validate(self):
        for block in self.blocks:
            assert block.dim == self.dim, "All blocks must have the same dim"
        return self

In [ ]:
FPN3DConfig.build([12, 24, 36], dim=24, interpolation_mode="test", kernel_size=3)


FPN3DConfig(
    blocks=[
        FPN3DBlockConfig(
            in_channels=None,
            out_channels=None,
            kernel_size=3,
            padding='same',
            stride=1,
            conv_kwargs={},
            transposed=False,
            normalization='batchnorm3d',
            normalization_pre_args=[],
            normalization_post_args=[],
            normalization_kwargs={},
            activation='relu',
            activation_kwargs={},
            sequence='CNA',
            drop_prob=0.0,
            dim=24,
            skip_conn_dim=12,
            is_deepest=False,
            interpolation_mode='test',
            merge_method='add'
        ),
        FPN3DBlockConfig(
            in_channels=None,
            out_channels=None,
            kernel_size=3,
            padding='same',
            stride=1,
            conv_kwargs={},
            transposed=False,
            normalization='batchnorm3d',
            normalization_pre_args=[],
           

# Architecture

### Basic block

In [ ]:
# | export


class FPN3DBlock(nn.Module):
    def __init__(self, config: FPN3DBlockConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__()

        self.config = FPN3DBlockConfig.model_validate(config | kwargs | {"kernel_size": 3})

        self.skip_conn_conv = CNNBlock3D(
            self.config,
            in_channels=self.config.skip_conn_dim,
            out_channels=self.config.dim,
            kernel_size=1,
            checkpointing_level=checkpointing_level,
        )

        if not self.config.is_deepest:
            self.out_conv = CNNBlock3D(
                self.config,
                in_channels=self.config.dim if self.config.merge_method == "add" else 2 * self.config.dim,
                out_channels=self.config.dim,
                checkpointing_level=checkpointing_level,
            )

        self.checkpointing_level2 = ActivationCheckpointing(2, checkpointing_level)

    def _forward(self, skip_conn_features: torch.Tensor, features: torch.Tensor | None):
        # skip_conn_features: (b, skip_conn_dim, d1, h1, w1)
        # features: (b, dim, d2, h2, w2)

        skip_conn_features = self.skip_conn_conv(skip_conn_features)
        # (b, dim, d1, h1, w1)

        if not self.config.is_deepest:
            features = F.interpolate(
                features,
                size=skip_conn_features.shape[2:],
                mode=self.config.interpolation_mode,
                align_corners=False,
            )
            # (b, dim, d1, h1, w1)

            if self.config.merge_method == "add":
                merged_features = skip_conn_features + features
                # (b, dim, d1, h1, w1)
            elif self.config.merge_method == "concat":
                merged_features = torch.cat((skip_conn_features, features), dim=1)
                # (b, 2 * dim, d1, h1, w1)

            merged_features = self.out_conv(merged_features)
            # (b, dim, d1, h1, w1)
        else:
            merged_features = skip_conn_features

        return merged_features

    def forward(self, *args, **kwargs):
        return self.checkpointing_level2(self._forward, *args, **kwargs)

In [ ]:
test = FPN3DBlock(dim=128, skip_conn_dim=256, is_deepest=True, checkpointing_level=2)
display(test)
display(test(torch.randn(2, 256, 4, 8, 8), None).shape)


FPN3DBlock(
  (skip_conn_conv): CNNBlock3D(
    (conv): Conv3d(256, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), padding=same, bias=False)
    (norm): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU()
    (checkpointing_level1): ActivationCheckpointing(enabled=True)
  )
  (checkpointing_level2): ActivationCheckpointing(enabled=True)
)

torch.Size([2, 128, 4, 8, 8])

In [ ]:
test = FPN3DBlock(dim=128, skip_conn_dim=256, is_deepest=False, checkpointing_level=1)
display(test)
display(test(torch.randn(2, 256, 8, 16, 16), torch.randn(2, 128, 4, 8, 8)).shape)


FPN3DBlock(
  (skip_conn_conv): CNNBlock3D(
    (conv): Conv3d(256, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), padding=same, bias=False)
    (norm): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU()
    (checkpointing_level1): ActivationCheckpointing(enabled=True)
  )
  (out_conv): CNNBlock3D(
    (conv): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=same, bias=False)
    (norm): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU()
    (checkpointing_level1): ActivationCheckpointing(enabled=True)
  )
  (checkpointing_level2): ActivationCheckpointing(enabled=False)
)

torch.Size([2, 128, 8, 16, 16])

### Complete architecture

In [ ]:
# | export


class FPN3D(nn.Module, PyTorchModelHubMixin):
    def __init__(self, config: FPN3DConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__()

        self.config = FPN3DConfig.model_validate(config | kwargs)

        self.blocks = nn.ModuleList()
        for i in range(len(self.config.blocks)):
            self.blocks.append(FPN3DBlock(self.config.blocks[i], checkpointing_level))

        self.checkpointing_level4 = ActivationCheckpointing(4, checkpointing_level)

    def _forward(self, features: list[torch.Tensor]):
        # features: [
        #   (b, in_dim1, d1, h1, w1),
        #   (b, in_dim2, d2, h2, w2),
        #   ...
        # ]

        outputs = [None]
        for i in reversed(range(len(features))):
            outputs.append(self.blocks[i](features[i], outputs[-1]))
        outputs = outputs[1:]

        return outputs

    def forward(self, *args, **kwargs):
        return self.checkpointing_level4(self._forward, *args, **kwargs)

In [ ]:
test_config = FPN3DConfig.build(skip_conn_dims=[64, 128, 256, 512], dim=128)
test_input = [
    torch.randn(2, 64, 16, 32, 32),
    torch.randn(2, 128, 8, 16, 16),
    torch.randn(2, 256, 4, 8, 8),
    torch.randn(2, 512, 2, 4, 4),
]
test = FPN3D(test_config, 3)

display(test)
display([output.shape for output in test(test_input)])


FPN3D(
  (blocks): ModuleList(
    (0): FPN3DBlock(
      (skip_conn_conv): CNNBlock3D(
        (conv): Conv3d(64, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), padding=same, bias=False)
        (norm): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU()
        (checkpointing_level1): ActivationCheckpointing(enabled=True)
      )
      (out_conv): CNNBlock3D(
        (conv): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=same, bias=False)
        (norm): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU()
        (checkpointing_level1): ActivationCheckpointing(enabled=True)
      )
      (checkpointing_level2): ActivationCheckpointing(enabled=True)
    )
    (1): FPN3DBlock(
      (skip_conn_conv): CNNBlock3D(
        (conv): Conv3d(128, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), padding=same, bias=False)
        (norm): BatchNorm3d(128, eps=1e-05, momentum=0.1,


[
    torch.Size([2, 128, 2, 4, 4]),
    torch.Size([2, 128, 4, 8, 8]),
    torch.Size([2, 128, 8, 16, 16]),
    torch.Size([2, 128, 16, 32, 32])
]

# nbdev

In [ ]:
!nbdev_export